In [1]:
import json
import requests
import urllib.request
import time
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

df = pd.DataFrame()
index = 0
last_date = ''

# 수정 변수들
coinName = "BTC"
to_date = '2021-06-01'
num = 30
scale = '1'
if num == 0:
    type = 'days'
else:
    type = 'minutes/{}'.format(num)


def get_coin_data_url(coinName, type, scale, cnt=400) :
    addr = 'https://crix-api-endpoint.upbit.com/v1/crix/candles/'
    if type == 'minutes' :
        basic_url = addr + type + '/' + scale + '?code=CRIX.UPBIT.KRW-' + coinName +'&count='+str(cnt)
    else :
        basic_url = addr + type + '/' + '?code=CRIX.UPBIT.KRW-' + coinName +'&count='+str(cnt)

    return basic_url

basic_url = get_coin_data_url(coinName,type,scale)
url = basic_url

def get_upbit_data(url, last_date, to_date) :
    global df
    global index

    fail2GetData = False
    response = ''
    while True:
        try :
            response = requests.get(url)
        except Exception as e:
            time.sleep(20)
            continue
        if str(response) == '<Response [200]>':
            break
        time.sleep(10)
    if ( fail2GetData )  :
        exit()

    data = response.json()

    if index == 0:
        df = pd.DataFrame(data)
        date = ''
        date = data[len(data)-1]['candleDateTime']
        index +=1

    else:
        df_for = pd.DataFrame(data)
        df = df.append(df_for,ignore_index=True)
        date = ''
        date = data[len(data)-1]['candleDateTime']


    return date




In [2]:

while (1) :
    last_date = get_upbit_data(url, last_date, to_date)
    tmp1 = last_date.split('T')
    if tmp1[0]  < to_date :
        break
    tmp2 = tmp1[1].split('+')
    target_date = tmp1[0] + ' ' + tmp2[0]
    url = basic_url + '&to=' + target_date    #to=2019-11-27 04:01:00
    time.sleep(2)

idx_todate = df.index[(df.candleDateTimeKst == '{}T00:00:00+09:00'.format(to_date))].tolist()[0]+1
idx_todate

df = df.drop(df.index[idx_todate:])
df

,code,candleDateTime,candleDateTimeKst,openingPrice,highPrice,lowPrice,tradePrice,candleAccTradeVolume,candleAccTradePrice,timestamp,unit
0,CRIX.UPBIT.KRW-BTC,2021-06-13T08:00:00+00:00,2021-06-13T17:00:00+09:00,41310000.0,41400000.0,41310000.0,41372000.0,34.298453,1.417585e+09,1623571482985,30
1,CRIX.UPBIT.KRW-BTC,2021-06-13T07:30:00+00:00,2021-06-13T16:30:00+09:00,41440000.0,41487000.0,41292000.0,41310000.0,279.043268,1.154530e+10,1623571199689,30
2,CRIX.UPBIT.KRW-BTC,2021-06-13T07:00:00+00:00,2021-06-13T16:00:00+09:00,41196000.0,41451000.0,41083000.0,41435000.0,322.375668,1.332370e+10,1623569400047,30
3,CRIX.UPBIT.KRW-BTC,2021-06-13T06:30:00+00:00,2021-06-13T15:30:00+09:00,40860000.0,41217000.0,40839000.0,41196000.0,139.388277,5.713153e+09,1623567599964,30
4,CRIX.UPBIT.KRW-BTC,2021-06-13T06:00:00+00:00,2021-06-13T15:00:00+09:00,40747000.0,40900000.0,40710000.0,40860000.0,82.713961,3.373840e+09,1623565799237,30
...,...,...,...,...,...,...,...,...,...,...,...
603,CRIX.UPBIT.KRW-BTC,2021-05-31T17:00:00+00:00,2021-06-01T02:00:00+09:00,43750000.0,43781000.0,43344000.0,43455000.0,94.137519,4.097559e+09,1622482195149,30
604,CRIX.UPBIT.KRW-BTC,2021-05-31T16:30:00+00:00,2021-06-01T01:30:00+09:00,43732000.0,43900000.0,43580000.0,43706000.0,113.171244,4.945885e+09,1622480397168,30
605,CRIX.UPBIT.KRW-BTC,2021-05-31T16:00:00+00:00,2021-06-01T01:00:00+09:00,43452000.0,44235000.0,43331000.0,43732000.0,514.176556,2.254488e+10,1622478600042,30
606,CRIX.UPBIT.KRW-BTC,2021-05-31T15:30:00+00:00,2021-06-01T00:30:00+09:00,43382000.0,43600000.0,43200000.0,43452000.0,160.459124,6.963492e+09,1622476798899,30


In [3]:
df['Date'] = 0
df['hour'] = 0
df['minute'] = 0

In [4]:
df['Date'][0] = df['candleDateTimeKst'][0].split('T')
df['Date'][0][0]


'2021-06-13'

In [5]:
for i in range(len(df)):
    df['Date'] [i] = df['candleDateTimeKst'][i].split('T')
    df['hour'] [i] = df['Date'][i][1].split('+')[0]
    df['minute'] [i] = df['hour'][i].split(':')[1]
    df['hour'] [i] = df['hour'][i].split(':')[0]
    df['Date'] [i] = df['Date'][i][0]


In [6]:
df

,code,candleDateTime,candleDateTimeKst,openingPrice,highPrice,lowPrice,tradePrice,candleAccTradeVolume,candleAccTradePrice,timestamp,unit,Date,hour,minute
0,CRIX.UPBIT.KRW-BTC,2021-06-13T08:00:00+00:00,2021-06-13T17:00:00+09:00,41310000.0,41400000.0,41310000.0,41372000.0,34.298453,1.417585e+09,1623571482985,30,2021-06-13,17,0
1,CRIX.UPBIT.KRW-BTC,2021-06-13T07:30:00+00:00,2021-06-13T16:30:00+09:00,41440000.0,41487000.0,41292000.0,41310000.0,279.043268,1.154530e+10,1623571199689,30,2021-06-13,16,30
2,CRIX.UPBIT.KRW-BTC,2021-06-13T07:00:00+00:00,2021-06-13T16:00:00+09:00,41196000.0,41451000.0,41083000.0,41435000.0,322.375668,1.332370e+10,1623569400047,30,2021-06-13,16,0
3,CRIX.UPBIT.KRW-BTC,2021-06-13T06:30:00+00:00,2021-06-13T15:30:00+09:00,40860000.0,41217000.0,40839000.0,41196000.0,139.388277,5.713153e+09,1623567599964,30,2021-06-13,15,30
4,CRIX.UPBIT.KRW-BTC,2021-06-13T06:00:00+00:00,2021-06-13T15:00:00+09:00,40747000.0,40900000.0,40710000.0,40860000.0,82.713961,3.373840e+09,1623565799237,30,2021-06-13,15,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
603,CRIX.UPBIT.KRW-BTC,2021-05-31T17:00:00+00:00,2021-06-01T02:00:00+09:00,43750000.0,43781000.0,43344000.0,43455000.0,94.137519,4.097559e+09,1622482195149,30,2021-06-01,02,0
604,CRIX.UPBIT.KRW-BTC,2021-05-31T16:30:00+00:00,2021-06-01T01:30:00+09:00,43732000.0,43900000.0,43580000.0,43706000.0,113.171244,4.945885e+09,1622480397168,30,2021-06-01,01,30
605,CRIX.UPBIT.KRW-BTC,2021-05-31T16:00:00+00:00,2021-06-01T01:00:00+09:00,43452000.0,44235000.0,43331000.0,43732000.0,514.176556,2.254488e+10,1622478600042,30,2021-06-01,01,0
606,CRIX.UPBIT.KRW-BTC,2021-05-31T15:30:00+00:00,2021-06-01T00:30:00+09:00,43382000.0,43600000.0,43200000.0,43452000.0,160.459124,6.963492e+09,1622476798899,30,2021-06-01,00,30


In [7]:

drops= ['Date','unit','candleAccTradePrice','openingPrice','code','candleDateTime','timestamp','candleDateTimeKst']
df=df.drop(drops,axis=1)
df=df[['hour','minute','tradePrice','highPrice','lowPrice','candleAccTradeVolume']]
df.head()

,hour,minute,tradePrice,highPrice,lowPrice,candleAccTradeVolume
0,17,0,41372000.0,41400000.0,41310000.0,34.298453
1,16,30,41310000.0,41487000.0,41292000.0,279.043268
2,16,0,41435000.0,41451000.0,41083000.0,322.375668
3,15,30,41196000.0,41217000.0,40839000.0,139.388277
4,15,0,40860000.0,40900000.0,40710000.0,82.713961


In [8]:
df['tradePrice'][0]

41372000.0

In [9]:
ups, downs = df['tradePrice'].loc[::-1].diff() ,df['tradePrice'].loc[::-1].diff()
# ups, downs = ups.loc[::-1],downs.loc[::-1]
ups[ups<0] =0
downs[downs>0] = 0
print(ups,downs)
# period = 14
# au = ups.ewm(com=period-1,min_periods=period).mean()
# ad = downs.ewm(com=period-1,min_periods=period).mean()
# au, ad = au.loc[::-1],ad.loc[::-1]
# RS = au/ad
# RSI = pd.Series(100-(100/(1+RS)))
# print(RSI)


607         NaN
606     52000.0
605    280000.0
604         0.0
603         0.0
         ...   
4      113000.0
3      336000.0
2      239000.0
1           0.0
0       62000.0
Name: tradePrice, Length: 608, dtype: float64 607         NaN
606         0.0
605         0.0
604    -26000.0
603   -251000.0
         ...   
4           0.0
3           0.0
2           0.0
1     -125000.0
0           0.0
Name: tradePrice, Length: 608, dtype: float64


In [10]:
period = 14 
au = ups.ewm(com = period-1, min_periods = period).mean()
ad = downs.abs().ewm(com = period-1, min_periods = period).mean() 
RS = au/ad 
RSI = pd.Series(100 - (100/(1+RS))) 
print(RSI)            

607          NaN
606          NaN
605          NaN
604          NaN
603          NaN
         ...    
4      42.034340
3      49.621378
2      54.212240
1      51.565542
0      52.796512
Name: tradePrice, Length: 608, dtype: float64
